# 랩미팅
goal.
1. Qtable을 dynamic하게 정의
2. Qtable을 학습시킨후 score에 대한 equation들을 출력
--- 현재 ---

3. 출력된 equation을 가지고 maude meta-level로 휴리스틱 서치

In [1]:
import maude
import random
import numpy as np
import numpy.ma as ma
import tqdm
from tqdm.notebook import tqdm

In [2]:
maude.init()
maude.load('./dp.maude')
m = maude.getCurrentModule()
print('Using', m, 'module')

Using DP5 module


In [5]:
def dp_generator():
    N = random.choice([5])
    P = [0] * N #np.zeros(N, dtype=int)
    C = [1] * N #np.ones(N, dtype=int)

    for i in range(N):
        c = random.randrange(3)
        if c == 1:
            # to left
            P[(i-1) % N] += 1
            C[i] = 0
        elif c == 2:
            # to right:
            P[i] += 1
            C[i] = 0

    # here, self.P[i] denotes the number of chopstics assigned for ith philos
    for i in range(N):
        if P[i] == 0:
            P[i] = random.randrange(2) # either think or hungry
        else:
            P[i] += 1 # one chopstick or eat
    
    s = []
    for i in range(N):
        if P[i] == 0:
            s.append(f'p({i},think)')
        elif P[i] == 1:
            s.append(f'p({i},hungry)')
        elif P[i] == 2:
            s.append(f'p({i},single)')
        elif P[i] == 3:
            s.append(f'p({i},eat)')
            
        if C[i] == 0:
            pass
        elif C[i] == 1:
            s.append(f'c({i})')
    return m.parseTerm(' || '.join(s))

dp_generator()

p(0, hungry) || c(0) || p(1, eat) || p(2, think) || p(3, single) || p(4, think) || c(4)

MaudeEnv does not know anything about the model in consideration

In [84]:
class MaudeEnv():
    def __init__(self, g, init_term=None):
        self.conf_gen = g
        self.reset(init_term)
        
    def reset(self, init_term=None):
        if init_term == None:
            t = self.conf_gen()
        else:
            t = init_term
        self.term = t
        self.acfg = self.get_acfg(t)
        self.nbrs = [(t,self.get_acfg(t)) for t,_,_,_ in t.search(1, m.parseTerm('X:Conf'), depth = 1)]  
        self.next_acfg = list(set([action for (_,action) in self.nbrs])) # remove duplicates
        return self.get_state() 
    
    def get_state(self):
        return {
            'term' : self.term,
            'acfg' : self.acfg,
            'nbrs' : self.nbrs,
            'next_acfg' : self.next_acfg,
        }
        
    def step(self, action):
        pairs = [(term, acfg) for (term,acfg) in self.nbrs if acfg == action]
        if pairs == []:
            raise Exception("invalid action")
        state = self.reset(random.choice(pairs)[0])
        done = True if self.nbrs == [] else False
        reward = 1 if done else 0
        return state, reward, done
    
    def get_acfg(self, t):
        acfg = m.parseTerm('aconf(' + t.prettyPrint(0) + ')') # TODO change aconf to acfg
        acfg.reduce()
        return acfg

In [133]:
class QTable():
    def __init__(self):
        self.q_init = 0
        self.q_dict = dict()
        
    def get_q(self, t1, t2):
        q_init = self.q_init
        #row = self.q_dict.get(t1, None)
        #if not row == None:
        if t1 in self.q_dict:
            return self.q_dict[t1].get(t2, q_init)
        return q_init
        
    def set_q(self, t1, t2, q):
        # TODO deepcopy terms
        if not t1 in self.q_dict:
            self.q_dict[t1] = { t2 : q }
        else:
            self.q_dict[t1][t2] = q
        
    def argmax_q(self, t1, nbrs): # nbrs: iterable if acfg's
        q_dict = self.q_dict
        if t1 in q_dict and len(nbrs) != 0:
            d = { t2 : q_dict[t1].get(t2, 0) for t2 in nbrs } # d = restriction of q_dict to tl
            return max(d, key=d.get)
        else:
            return -1
        
    def max_q(self, t):
        q_dict = self.q_dict
        if t in q_dict: # assume q_dict[t] is nonempty
            return max(q_dict[t].values())
        return 0
    
    def get_size(self):
        # returns the number of nonzero entries in the QTable
        ret = 0
        for _, d in self.q_dict.items():
            ret += len(d)
        return ret
    
    def print_v(self):
        q_dict = self.q_dict
        print(f'fmod SCORE is')
        for t in q_dict:
            print(f'  eq score({t}) = {self.max_q(t)} .')
        print(f'  eq score(X) = {self.q_init} [owise] .')
        print(f'endfm')        
    
    def print_q(self):
        q_dict = self.q_dict
        print('load dp.maude')
        print('mod SCORE is')
        print('  pr DP5 .')
        print('  pr FLOAT .')
        print('  op score : AConf AConf -> Float .')
        for t1, d in q_dict.items():
            for t2, q in d.items():
                print(f'  eq score({t1}, {t2}) = {q} .')
        print(f'  eq score(X:AConf, Y:AConf) = {self.q_init} [owise] .') # TODO: 0 should be printed 0.0
        print(f'endm')

In [134]:
def greedy_policy(Qt, state):
    # Exploitation: take the action with the highest state, action value
    t = state["acfg"]
    nbrs = state["next_acfg"]
    return Qt.argmax_q(t,nbrs)

def eps_greedy_policy(Qtable, state, epsilon):
    random_num = random.uniform(0, 1)
    if random_num > epsilon: # exploitation
        return greedy_policy(Qtable, state)
    else: # exploration
        nbrs = state["next_acfg"]
        if len(nbrs) != 0:
            return random.choice(nbrs)
        else:
            return -1

def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, qt):
    for episode in tqdm(range(n_training_episodes)):
        # Reduce epsilon (because we need less and less exploration)
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)
        # Reset the environment
        state = env.reset()
        step = 0
        done = False

        # repeat
        for step in range(max_steps):
            # Choose the action At using epsilon greedy policy
            s = state["acfg"]
            a = eps_greedy_policy(qt, state, epsilon)
            
            # assert action not -1
            if type(a) == type(-1):
                break

            # Take action At and observe Rt+1 and St+1
            # Take the action (a) and observe the outcome state(s') and reward (r)
            #print('episode:', episode, 'step:', step, 'a:',a)
            new_state, reward, done = env.step(a)

            # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]

            #Qtable[s][a] = Qtable[s][a] + learning_rate * (
            #    reward + gamma * np.max(Qtable[a]) - Qtable[s][a]
            #)
            
            new_q = qt.get_q(s, a) + learning_rate * (
                reward + gamma * qt.max_q(a) - qt.get_q(s, a)
            )
            qt.set_q(s, a, new_q)

            # If terminated or truncated finish the episode
            if done:
                break

            # Our next state is the new state
            state = new_state
    print('training done!')
    return qt

In [122]:
# Training parameters
n_training_episodes = 100  # Total training episodes
learning_rate = 0.7  # Learning rate

# Evaluation parameters
n_eval_episodes = 100  # Total number of test episodes

# Environment parameters
#env_id = "FrozenLake-v1"  # Name of the environment
max_steps = 300  # Max steps per episode
gamma = 0.95  # Discounting rate
eval_seed = []  # The evaluation seed of the environment

# Exploration parameters
max_epsilon = 1.0  # Exploration probability at start
min_epsilon = 0.05  # Minimum exploration probability
decay_rate = 0.0005  # Exponential decay rate for exploration prob

In [135]:
# train Qtable
env = MaudeEnv(dp_generator)
Qtable = QTable()
Qtable = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable)

  0%|          | 0/100 [00:00<?, ?it/s]

training done!


In [137]:
Qtable.print_q()

load dp.maude
mod SCORE is
  pr DP5 .
  pr FLOAT .
  op score : AConf AConf -> Float .
  eq score(aconf(p(--, think) || p(--, hungry) || p(--, eat)), aconf(p(--, think) || p(--, hungry) || p(--, eat))) = 0.8145062499999999 .
  eq score(aconf(p(--, think) || p(--, hungry) || p(--, eat)), aconf(p(--, hungry) || p(--, eat))) = 0.8145062499999999 .
  eq score(aconf(p(--, think) || p(--, hungry) || p(--, eat)), aconf(p(--, think) || p(--, hungry))) = 0.8573749999999999 .
  eq score(aconf(p(--, think) || p(--, hungry) || p(--, eat)), aconf(p(--, think) || p(--, hungry) || p(--, single) || p(--, eat))) = 0.8573749999999999 .
  eq score(aconf(p(--, think) || p(--, hungry) || p(--, eat)), aconf(p(--, think) || p(--, single) || p(--, eat))) = 0.8573749999999999 .
  eq score(aconf(p(--, hungry) || p(--, eat)), aconf(p(--, think) || p(--, hungry))) = 0.8573749999999999 .
  eq score(aconf(p(--, hungry) || p(--, eat)), aconf(p(--, hungry) || p(--, single) || p(--, eat))) = 0.8573749999999999 .
  eq 

In [118]:
qd = Qtable.q_dict
s = 0
for t1, d in qd.items():
    for t2, q in d.items():
        print(f'eq score({t1}, {t2}) = {q}')

eq score(aconf(p(--, hungry) || p(--, single) || p(--, eat)), aconf(p(--, think) || p(--, hungry) || p(--, single))) = 0.9025
eq score(aconf(p(--, hungry) || p(--, single) || p(--, eat)), aconf(p(--, hungry) || p(--, single) || p(--, eat))) = 0.8573749999999999
eq score(aconf(p(--, hungry) || p(--, single) || p(--, eat)), aconf(p(--, think) || p(--, hungry) || p(--, single) || p(--, eat))) = 0.8573749999999999
eq score(aconf(p(--, hungry) || p(--, single) || p(--, eat)), aconf(p(--, hungry) || p(--, eat))) = 0.8145062499999999
eq score(aconf(p(--, think) || p(--, hungry) || p(--, single)), aconf(p(--, think) || p(--, hungry) || p(--, single) || p(--, eat))) = 0.8573749999999999
eq score(aconf(p(--, think) || p(--, hungry) || p(--, single)), aconf(p(--, think) || p(--, single))) = 0.9025
eq score(aconf(p(--, think) || p(--, hungry) || p(--, single)), aconf(p(--, hungry) || p(--, single))) = 0.95
eq score(aconf(p(--, think) || p(--, hungry) || p(--, single)), aconf(p(--, think) || p(--, 

In [91]:
# test for QTable

env = MaudeEnv(dp_generator, m.parseTerm('c(2) || c(3) || p(0, single) || p(1, think) || p(2, hungry) || p(3, hungry) || p(4, eat)'))

t = env.acfg
t1 = m.parseTerm('aconf(p(--, think) || p(--, single) || p(--, hungry))') # hit
t2 = m.parseTerm('aconf(p(--, think) || p(--, hungry) || p(--, single) || p(--, eat))') # hit
t3 = m.parseTerm('aconf(p(--, hungry) || p(--, single) || p(--, eat))') # miss
t4 = m.parseTerm('aconf(p(--, think))') # invalid move
t1.reduce()
t2.reduce()
t3.reduce()
t4.reduce()

qt = QTable()
qt.set_q(t, t1, 2)
qt.set_q(t, t2, 3)
qt.set_q(t, t4, 1)

#print(env.next_acfg)
for next_t in env.next_acfg:
    print('next:', next_t)
    print('result:',qt.get_q(t,next_t))
    
print('max:', qt.max_q(t))
print('argmax:', qt.argmax_q(t, []))

next: aconf(p(--, think) || p(--, hungry) || p(--, single))
result: 2
next: aconf(p(--, think) || p(--, hungry) || p(--, single) || p(--, eat))
result: 3
next: aconf(p(--, hungry) || p(--, single) || p(--, eat))
result: 0
max: 3
argmax: -1


False

In [37]:
env = MaudeEnv(dp_generator, m.parseTerm('c(2) || c(3) || p(0, single) || p(1, think) || p(2, hungry) || p(3, hungry) || p(4, eat)'))

#print(env.get_state())

t = env.acfg
t1 = m.parseTerm('aconf(p(--, think) || p(--, hungry) || p(--, single))') # hit
t2 = m.parseTerm('aconf(p(--, think) || p(--, hungry) || p(--, single) || p(--, eat))') # hit
t3 = m.parseTerm('aconf(p(--, hungry) || p(--, single) || p(--, eat))') # miss
t4 = m.parseTerm('aconf(p(--, think))') # invalid move
t1.reduce()
t2.reduce()
t3.reduce()
t4.reduce()
Qtable = { t : { t1: 2, t2: 3, t4: 1} }

for next_t in env.next_acfg:
    print('next:', next_t)
    print('result:',Qtable[t].get(next_t, 0))

'''
d1 = Qtable[t]
d2 = dict.fromkeys(env.next_acfg, 0)
{**d1, **d2}
print(Qtable[t])
print(env.next_acfg)
{ k: Qtable[t].get(k, 42) for k in env.next_acfg }

for k in Qtable[t]:
    print(k, k.__hash__())
    
print('===')

for k in env.next_acfg:
    print(k, k.__hash__())
'''


#print(dict(Qtable[t], **))
#max(Qtable[t], key=Qtable[t].get)
#[ dict.get(k, my_default_value) for k in my_iterable ]

next: aconf(p(--, think) || p(--, hungry) || p(--, single))
result: 2
next: aconf(p(--, think) || p(--, hungry) || p(--, single) || p(--, eat))
result: 3
next: aconf(p(--, hungry) || p(--, single) || p(--, eat))
result: 0


"\nd1 = Qtable[t]\nd2 = dict.fromkeys(env.next_acfg, 0)\n{**d1, **d2}\nprint(Qtable[t])\nprint(env.next_acfg)\n{ k: Qtable[t].get(k, 42) for k in env.next_acfg }\n\nfor k in Qtable[t]:\n    print(k, k.__hash__())\n    \nprint('===')\n\nfor k in env.next_acfg:\n    print(k, k.__hash__())\n"

In [22]:
t = m.parseTerm('c(2) || c(3) || p(0, single) || p(1, think) || p(2, hungry) || p(3, hungry) || p(4, eat)')
#print(t.__hash__())
t = m.parseTerm('aconf(c(2) || c(3) || p(0, single) || p(1, think) || p(2, hungry) || p(3, hungry) || p(4, eat))')
#print(t.__hash__())
t.reduce()
#print(t.__hash__())
print('t:', t, t.__hash__())
tt = m.parseTerm('aconf(p(--, think) || p(--, hungry) || p(--, single) || p(--, eat))')
tt.reduce()
ttt = m.parseTerm('aconf(p(--, think) || p(--, hungry) || p(--, eat) || p(--, single))')
print('tt:', tt, tt.__hash__())
print('ttt:', ttt, ttt.__hash__())
print('equal:', tt.equal(t))
s = 'aconf(c(2) || c(3) || p(0, single) || p(1, think) || p(2, hungry) || p(3, hungry) || p(4, eat))'
ss = 'aconf(p(--, think) || p(--, hungry) || p(--, single) || p(--, eat))'
sss = 'aconf(p(--, think) || p(--, hungry) || p(--, eat) || p(--, single))'
eq = m.parseTerm(ss + ' == ' + sss)
eq.reduce()
print(eq)

t: aconf(p(--, think) || p(--, hungry) || p(--, single) || p(--, eat)) 14293894632505905964
tt: aconf(p(--, think) || p(--, hungry) || p(--, single) || p(--, eat)) 14293894632505905964
ttt: aconf(p(--, think) || p(--, hungry) || p(--, single) || p(--, eat)) 1443846585
equal: True
true


In [27]:
t1 = m.parseTerm('aconf(p(--, think) || p(--, hungry) || p(--, single))')
t2 = m.parseTerm('aconf(p(--, hungry) || p(--, think) || p(--, single))')
t3 = m.parseTerm('p(0, think) || p(1, hungry) || p(2, single)')
t4 = m.parseTerm('p(1, hungry) || p(0, think) || p(2, single)')
print(t1.equal(t2))
print(t3.equal(t4))
print(t1.hash())
print(t2.hash())
print(t3.hash())
print(t4.hash()) # why equal() returns false while hash vals are the same?
my_dict = {}
my_dict[t3] = 42
print(my_dict[t4])

False
False
3315544020
3315544020
3364720003
3364720003
42


In [5]:
def abs_greedy_policy(Qtable, state):
    # Exploitation: take the action with the highest state, action value
    aidx = state["aidx"]
    mask = state["mask"]
    #mask = np.where(True, mask^1, mask) # flip 0 & 1
    if 0 in mask:
        masked_Q = ma.masked_array(Qtable[aidx][:], mask=mask)
        action = np.argmax(masked_Q)
    else:
        action = -1 # deadlock
    return action

def abs_epsilon_greedy_policy(Qtable, state, epsilon):
    # Randomly generate a number between 0 and 1
    random_num = random.uniform(0, 1)
    # if random_num > greater than epsilon --> exploitation
    if random_num > epsilon:
        # Take the action with the highest value given a state
        # np.argmax can be useful here
        action = abs_greedy_policy(Qtable, state)
    # else --> exploration
    else:
        nbrs = state["nbrs"]
        if nbrs != []:
            action = random.choice(nbrs)[1]
        else:
            action = -1
    return action

def abs_train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
    for episode in tqdm(range(n_training_episodes)):
        # Reduce epsilon (because we need less and less exploration)
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)
        # Reset the environment
        state = env.reset()
        step = 0
        done = False

        # repeat
        for step in range(max_steps):
            # Choose the action At using epsilon greedy policy
            s = state["aidx"]
            a = abs_epsilon_greedy_policy(Qtable, state, epsilon)
            
            # assert action not -1
            if a == -1:
                break

            # Take action At and observe Rt+1 and St+1
            # Take the action (a) and observe the outcome state(s') and reward (r)
            new_state, reward, done = env.step(a)

            # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]

            Qtable[s][a] = Qtable[s][a] + learning_rate * (
                reward + gamma * np.max(Qtable[a]) - Qtable[s][a]
            )

            # If terminated or truncated finish the episode
            if done:
                break

            # Our next state is the new state
            state = new_state
    print('training done!')
    return Qtable

In [7]:
env = MaudeEnv(dp_generator)
Qtable = np.zeros(shape=(16,16))
Qtable = abs_train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable)

AttributeError: 'NoneType' object has no attribute 'reduce'

In [ ]:
print('absQtable density:', np.count_nonzero(Qtable), '/', Qtable.size)
print(Qtable.shape)
print(Qtable)

In [ ]:
# e-greedy Simulator
def abs_simulate(Qtable, max_steps=1000, epsilon=0.01):
    state = env.reset()

    for step in range(max_steps):
        # Choose the action At using epsilon greedy policy
        action = abs_epsilon_greedy_policy(Qtable, state, epsilon)
        if action == -1:
            break

        # Take action At and observe Rt+1 and St+1
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done = env.step(action)

        # If terminated or truncated finish the episode
        if done:
            break

        # Our next state is the new state
        state = new_state
        
    return step + 1

In [ ]:
max_steps = 1000
res_trained = []
res_random = []
for _ in tqdm(range(100)):
    num_steps_trained = abs_simulate(Qtable, max_steps=max_steps, epsilon=0)
    num_steps_random = abs_simulate(Qtable, max_steps=max_steps, epsilon=1)
    res_trained.append(num_steps_trained)
    res_random.append(num_steps_random)
print('trained:')
print(res_trained)
print('avg:')
print(np.average(res_trained))
print('=====')
print('random:')
print(res_random)
print('avg:')
print(np.average(res_random))

In [ ]:
env = MaudeEnv(dp_generator)
env.mask

In [ ]:
Qtable = np.zeros(shape=(16,16))
print(Qtable)

In [ ]:
#t = m.parseTerm("p(0,think) || c(0) || p(s(0),think) || c(s(0)) || p(s(s(0)),think) || c(s(s(0)))")
env = MaudeEnv(dp_generator)

while env.available_actions != []:
    print('term:', env.term)
    #print('aidx:', env.aidx)
    action = random.choice(env.available_actions)
    print('action (next aidx):', action)
    state, reward, done = env.step(action)
    print(f'action: {action}, state: {state}, reward: {reward}, done: {done}')
    print('===')
print('last term:', env.term)

In [ ]:
tterm = m.parseTerm('p(0, single) || p(1, hungry) || p(2, eat) || p(3, hungry) || p(4, eat)')
print(tterm)
tenv = MaudeEnv(dp_generator, tterm)
state = tenv.reset(tterm)
#print(state)
a = abs_greedy_policy(Qtable, state)
print(a)
state, reward, done = tenv.step(a)
print(state, done)
#print(abs_epsilon_greedy_policy(Qtable, state, 0.5))

In [11]:
t = m.parseTerm('p(0, hungry) || c(0) || p(1, eat) || p(2, think) || p(3, single) || p(4, think) || c(4)')
v = m.parseTerm('C:Conf')
print('current term:',t)
i = 0
for r, sb, ctx, rl in t.apply('th'):
    print('')
    print(f'=== Action {i} of applying rule [th] ===')
    print(f'< RULE: {rl}, SUBSTITUTION: {sb} >')
    print(f'--- abstract action --->')
    asb = ''
    for (var,term) in sb:
        sort = str(var.getSort())
        alpha = 'alpha-' + sort # alpha-Nat
        aterm = m.parseTerm(f'{alpha}({str(term)})') # alpha-Nat(term)
        aterm.reduce() # unit
        asb += ('I:One=' + str(aterm))
    print(f'< RULE: {rl}, SUBSTITUTION: {asb} >')
    i += 1

current term: p(0, hungry) || c(0) || p(1, eat) || p(2, think) || p(3, single) || p(4, think) || c(4)

=== Action 0 of applying rule [th] ===
< RULE: rl [th] : p(I:Nat, think) => p(I:Nat, hungry) ., SUBSTITUTION: I:Nat=2 >
--- abstract action --->
< RULE: rl [th] : p(I:Nat, think) => p(I:Nat, hungry) ., SUBSTITUTION: I:One=unit >

=== Action 1 of applying rule [th] ===
< RULE: rl [th] : p(I:Nat, think) => p(I:Nat, hungry) ., SUBSTITUTION: I:Nat=4 >
--- abstract action --->
< RULE: rl [th] : p(I:Nat, think) => p(I:Nat, hungry) ., SUBSTITUTION: I:One=unit >
